In [5]:
import random
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

#Практика 4.10


In [7]:
# Функция для генерации равномерного распределения
def create_uniform_sequence(seq_len, min_val = 0, max_val = 1):
    """ Создает последовательность случайных чисел с равномерным распределением """
    result_sequence = [random.uniform(min_val, max_val) for _ in range(seq_len)]
    return result_sequence

# Функция для генерации суммы равномерных распределенных чисел
def create_sum_uniform_sequence(seq_len, min_val, max_val, num_sum):
    """ Создает последовательность сумм чисел с равномерным распределением """
    result_sequence = [sum(random.uniform(min_val, max_val) for _ in range(num_sum)) for _ in range(seq_len)]
    return result_sequence

# Функция для генерации нормального распределения
def create_normal_sequence(seq_len, avg=0, std_dev=1):
    """ Создает последовательность случайных чисел с нормальным распределением """
    result_sequence = [random.gauss(avg, std_dev) for _ in range(seq_len)]
    return result_sequence

# Функция для генерации распределения Коши
def create_cauchy_sequence(seq_len, loc=0, scale_factor=1):
    """ Создает последовательность случайных чисел с распределением Коши """
    result_sequence = stats.cauchy.rvs(loc=loc, scale=scale_factor, size=seq_len)
    return result_sequence

# Функция для расчета медианной абсолютной девиации (MAD)
def calculate_mad(data):
    """ Вычисляет MAD для переданных данных """
    median = np.median(data)  # Вычисляем медиану
    mad = np.median(np.abs(data - median))  # Вычисляем MAD
    return mad

# Функция для вычисления доверительных интервалов для MAD
def confidence_interval_mad(data, confidence=0.95):
    """ Вычисляет доверительный интервал для MAD """
    mad = calculate_mad(data)  # Рассчитываем MAD
    n = len(data)  # Размер выборки
    # Используем теорему центральной предельной для оценки стандартной ошибки
    se = mad / np.sqrt(n)  # Стандартная ошибка для MAD
    # Находим критические значения
    z_score = stats.norm.ppf((1 + confidence) / 2)  # z-значение для заданного уровня доверия
    return mad - z_score * se, mad + z_score * se  # Возвращаем доверительный интервал

In [8]:
sample_sizes = [50, 100, 500, 1000]  # Разные размеры выборок
distributions = {
        "Uniform": create_uniform_sequence,
        "Sum of Uniform": create_sum_uniform_sequence,
        "Normal": create_normal_sequence,
        "Cauchy": create_cauchy_sequence
    }

    # Генерируем данные для заданного размера выборки 10^7 для оценки MAD
true_value_size = 10**7
true_value_uniform = calculate_mad(create_uniform_sequence(true_value_size, 0, 1))
true_value_normal = calculate_mad(create_normal_sequence(true_value_size))
true_value_cauchy = calculate_mad(create_cauchy_sequence(true_value_size))

    # Исследуем доверительные интервалы для каждой распредения
for dist_name, dist_func in distributions.items():
  print(f"Распространение: {dist_name}")

  for size in sample_sizes:
    if dist_name == "Sum of Uniform":
      data = dist_func(size, 0, 1, 10)  # Генерация данных с суммами
    else:
      data = dist_func(size)  # Генерация данных для других распределений

            # Рассчитываем доверительный интервал для MAD
    ci = confidence_interval_mad(data)
    print(f"Размер выборки: {size},  Доверительный интервал для MAD: {ci}")

    print("")  # Разрыв строки для удобства чтения


Распространение: Uniform
Размер выборки: 50,  Доверительный интервал для MAD: (0.14496876926852403, 0.2561516277071825)

Размер выборки: 100,  Доверительный интервал для MAD: (0.17519773865025268, 0.2606155793830749)

Размер выборки: 500,  Доверительный интервал для MAD: (0.23724600018209518, 0.2828320100661782)

Размер выборки: 1000,  Доверительный интервал для MAD: (0.23221104065223006, 0.2628976301112415)

Распространение: Sum of Uniform
Размер выборки: 50,  Доверительный интервал для MAD: (0.3921457206162296, 0.6928993406034452)

Размер выборки: 100,  Доверительный интервал для MAD: (0.5143926076681126, 0.7651852615827274)

Размер выборки: 500,  Доверительный интервал для MAD: (0.5961922306249164, 0.7107485346183274)

Размер выборки: 1000,  Доверительный интервал для MAD: (0.6133441460028247, 0.6943973119186716)

Распространение: Normal
Размер выборки: 50,  Доверительный интервал для MAD: (0.5718346839604204, 1.010399590304636)

Размер выборки: 100,  Доверительный интервал для MAD:

# Домашнее задание

In [11]:
# Номер 3

# Функция для оценки размера выборки по итерационной процедуре
#Итерационная процедура (estimate_sample_size_iterative): мы продолжаем генерировать
#выборки и учитывать средние значения, пока они не приблизятся к истинному среднему более чем на дельту
def estimate_sample_size_iterative(target_mean, delta, distribution_func, iterations=1000):
    samples = []
    sample_means = []

    while True:
        sample = distribution_func(100)  # Берем фиксированный начальный размер выборки
        samples.append(sample)
        sample_mean = np.mean(sample)
        sample_means.append(sample_mean)

        # Вычисление стандартной ошибки
        se = np.std(sample) / np.sqrt(len(sample))

        # Проверка условия остановки
        if abs(sample_mean - target_mean) < delta:
            break

    return len(samples), sample_means

# Функция для оценки размера выборки по оценке сверху
#Оценка сверху (estimate_sample_size_upper_bound): мы используем стандартное отклонение и точность (определяется
#дельтой) для расчета необходимого размера выборки с использованием неравенства Чебышёва.
def estimate_sample_size_upper_bound(delta, distribution_func, z_value=1.96, iterations=1000):
    # z_value = 1.96 для 95% доверительного интервала
    return (z_value * 1 / delta) ** 2  # Здесь 1 - ошибка выборки, когда распределение нормальное

# Основной код для тестирования
true_mean_uniform = (0 + 1) / 2
true_mean_normal = 0
true_mean_cauchy = np.mean(np.random.standard_cauchy(size=10000))  # Примерный расчет

deltas = [0.1, 0.01, 0.001]

results_iterative = {'Uniform': [], 'Normal': [], 'Cauchy': []}
results_upper_bound = {'Uniform': [], 'Normal': [], 'Cauchy': []}

for delta in deltas:
    n_iter_uniform, _ = estimate_sample_size_iterative(true_mean_uniform, delta, create_uniform_sequence)
    n_iter_normal, _ = estimate_sample_size_iterative(true_mean_normal, delta, create_normal_sequence)
    n_iter_cauchy, _ = estimate_sample_size_iterative(true_mean_cauchy, delta, create_cauchy_sequence)

    results_iterative['Uniform'].append(n_iter_uniform)
    results_iterative['Normal'].append(n_iter_normal)
    results_iterative['Cauchy'].append(n_iter_cauchy)

    n_upper_uniform = estimate_sample_size_upper_bound(delta, create_uniform_sequence)
    n_upper_normal = estimate_sample_size_upper_bound(delta, create_normal_sequence)
    n_upper_cauchy = estimate_sample_size_upper_bound(delta, create_cauchy_sequence)

    results_upper_bound['Uniform'].append(n_upper_uniform)
    results_upper_bound['Normal'].append(n_upper_normal)
    results_upper_bound['Cauchy'].append(n_upper_cauchy)

# Вывод результатов
print("Итерационная процедура:")
print("Delta\tUniform\tNormal\tCauchy")
for i, delta in enumerate(deltas):
    print(f"{delta}\t{results_iterative['Uniform'][i]}\t{results_iterative['Normal'][i]}\t{results_iterative['Cauchy'][i]}")

print("\nОценка сверху:")
print("Delta\tUniform\tNormal\tCauchy")
for i, delta in enumerate(deltas):
    print(f"{delta}\t{results_upper_bound['Uniform'][i]:.2f}\t{results_upper_bound['Normal'][i]:.2f}\t{results_upper_bound['Cauchy'][i]:.2f}")

Итерационная процедура:
Delta	Uniform	Normal	Cauchy
0.1	1	2	6
0.01	3	3	26
0.001	81	100	1500

Оценка сверху:
Delta	Uniform	Normal	Cauchy
0.1	384.16	384.16	384.16
0.01	38416.00	38416.00	38416.00
0.001	3841600.00	3841600.00	3841600.00
